In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp
import threading

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
            x = requests.get(uri).text
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            uri = 'https://' + '/'.join(mod[:-1]) + '/raw/main/' + mod[-1] + '.py'
            x = requests.get(uri).text
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except Exception as err:
        print(err)

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()

def img2b64(x):
    if isinstance(x, bytes):
        return base64.b64encode(x).decode()
    elif isinstance(x, str):
        with open(x, 'rb') as fr:
            return base64.b64encode(fr.read()).decode()
    raise


CPython 3.6.9
IPython 7.16.1

numpy 1.19.4
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed


## 引入工具

In [2]:
_IMPORT_('import gitee.com/qrsforever/nb_easy/easy_widget')

## 配置界面

In [3]:
RACEURL = 'http://116.85.9.170:9119'
API_INFERENCE = f'{RACEURL}/raceai/framework/inference'
API_POPMSG = f'{RACEURL}/raceai/private/popmsg'
MODEL_TASK = 'zmq.repnet_tf.inference'
MSGKEY = 'zmq.repnet_tf.test'

# 00cf60a8f512-20210719104431.mp4: [0.24, 0.05, 0.354, 0.225]

play_list = [
    ('焊接', 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet/20210720102238.mp4'),
    ('4G_1', 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet/4G_1.mp4'),
    ('4G_2', 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet/4G_2.mp4'),
    ('切割', 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet/20210720143047.mp4'),
]

schema = {
    'type': 'page',
    'objs': [
        {
            'type': 'tab',
            'objs': [
                {
                    'name': 'CFG',
                    'objs': [
                        {
                            'type': 'H',
                            'objs': [
                                nbeasy_widget_string('task', 'Task', MODEL_TASK, width=400, readonly=True),
                                nbeasy_widget_string('cfg.pigeon.msgkey', 'Mssage Key', MSGKEY, readonly=True),
                                nbeasy_widget_string('cfg.pigeon.user_code', 'User Code', 'notebook', readonly=True),
                            ]
                        },
                        {
                            'type': 'accordion',
                            'objs': [
                                {
                                    'name': 'Video Configuration',
                                    'objs': [
                                        nbeasy_widget_stringenum('_cfg.video', 'Video', 0, enums=play_list, width=400),
                                        {
                                            'type': 'V',
                                            'objs': [
                                                nbeasy_widget_bytes('cfg.video', '', 'http://', width=620),
                                                nbeasy_widget_video('_cfg.play_video', 'Play Video', '', format='url', width=720, height=480)
                                            ]
                                        },
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_booltrigger('cfg.best_stride_video', 'Save Stride Video', True, trigger=[
                                                    nbeasy_widget_bool('cfg.osd_sims', 'OSD Sims', True),
                                                ]),
                                                nbeasy_widget_bool('cfg.save_video', 'Save Full Video', False),
                                            ]
                                        },
                                    ]
                                },
                                {
                                    'name': 'Score Configuration',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_bool('cfg.constant_speed', 'Constant Speed', False),
                                                nbeasy_widget_bool('cfg.median_filter', 'Median Filter', True),
                                                nbeasy_widget_bool('cfg.fully_periodic', 'Fully Periodic', False),
                                            ]
                                        }
                                    ]
                                },
                                {
                                    'name': 'Algo Configuration',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_int('cfg.batch_size', 'Batch Size', 20),
                                                nbeasy_widget_intarray('cfg.strides', 'Strides', [3, 5, 7, 9, 11, 13], width=400),
                                            ]
                                        }
                                    ]
                                },
                                {
                                    'name': 'Optimize Configuration',
                                    'objs': [
                                        {
                                            'type': 'H',
                                            'objs': [
                                                nbeasy_widget_booltrigger('cfg.rm_still', 'Remove Still', True, trigger=[
                                                    nbeasy_widget_float('cfg.area_rate_threshold', 'Area Threshold', 0.003, min_=0.00001, max_=0.005),
                                                ]),
                                                nbeasy_widget_booltrigger('_cfg.focus_box', 'Focus Box', False, trigger=[
                                                    nbeasy_widget_floatarray('cfg.focus_box', 'Rect', [0], width=300),
                                                ]),
                                                nbeasy_widget_booltrigger('_cfg.black_box', 'Black Box', False, trigger=[
                                                    {
                                                        'type': 'H',
                                                        'objs': [
                                                            nbeasy_widget_floatarray('cfg.black_box', 'Rect', [0], width=300),
                                                            nbeasy_widget_bool('cfg.black_overlay', 'Overlay', True)
                                                        ]
                                                    }
                                                ]),
                                            ]
                                        },
                                        nbeasy_widget_float('cfg.eta', 'Eta Rate', 1.0),
                                    ]
                                }
                            ]
                        }, # end accordion
                    ]
                },
                {
                    'name': 'RUN',
                    'objs': [
                        {
                            'type': 'H',
                            'objs': [
                                nbeasy_widget_button('_cfg.btn_start', 'Start', handler='start_inference', width=100, style='primary'),
                                nbeasy_widget_button('_cfg.btn_stop', 'Stop', handler='stop_inference', width=100, style='primary'),
                                nbeasy_widget_progressbar('_cfg.progressbar', 'Progress:', style='success', width=600)
                            ]
                        },
                        {
                            'type': 'H',
                            'objs': [
                                nbeasy_widget_text('_cfg.result_output', '', width=500, height=480),
                                nbeasy_widget_video('_cfg.play_stride_video', 'Stride Video', '', format='url', width=720, height=480),
                            ]
                        }
                    ]
                }
            ]
        }, # end tab
        {
            'type': 'jsdlink',
            'objs': [
                {
                    'source': ['_cfg.video', 'value'],
                    'target': ['cfg.video', 'value']
                },
                {
                    'source': ['cfg.video', 'bvalue'],
                    'target': ['_cfg.play_video', 'value'],
                }
            ] 
        }, # end jslink
        {
            'type': 'interactive',
            'objs': [
                {
                    'handler': 'xxx',
                    'params': {
                        'arg1': 'cfg.focus_box',
                        'arg2': 'cfg.black_box',
                    }
                }
            ]
            
        }, # end interactive
    ]
}

In [4]:
def start_inference(btn):
    bar = btn.context.get_widget_byid('_cfg.progressbar')
    out = btn.context.get_widget_byid('_cfg.result_output')
    psv = btn.context.get_widget_byid('_cfg.play_stride_video')
    
    reqdata = btn.context.get_all_json()
    requests.get(url=f'{API_POPMSG}?key={MSGKEY}')
    result = json.loads(requests.post(url=API_INFERENCE, json=reqdata).text)
    out.value = json.dumps(reqdata, indent=4)
    if 'errno' in result:
        if result['errno'] < 0:
            out.value = json.dumps(result, indent=4)
            return
        
    btn.disabled = True
    def _run_result(btn, bar, out, psv):
        cur_try = 0
        err_max = 20
        while cur_try < err_max:
            result = json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
            if len(result) == 0:
                time.sleep(1)
                cur_try += 1
                continue
            result = result[-1]
            cur_try = 0
            if result['errno'] != 0:
                out.value = json.dumps(result, indent=4)
                bar.value = 100.0
                break
            bar.value = int(result['progress'])
            if bar.value == 100.0:
                out.value += '\n' + json.dumps(result, indent=4)
                psv.value = result['stride_mp4'].encode()
        btn.disabled = False
    threading.Thread(target=_run_result, kwargs={
        'btn': btn,
        'bar': bar,
        'out': out,
        'psv': psv
    }).start()
    
def stop_inference(btn):
    pass
   

events = {
    'start_inference': start_inference,
    'stop_inference': stop_inference,
}

easy = nbeasy_schema_parse(schema, events=events, debug=True);

Box(children=(Box(children=(VBox(children=(Tab(children=(VBox(children=(HBox(children=(Text(value='zmq.repnet_…